In [1]:
import pandas as pd
from transformers import BertTokenizer,BertModel, AddedToken, AutoModel, AutoTokenizer

In [2]:
DATA_PATH = 'datasets/'
ADDITIONAL_DATA = 'additional_data/'

In [3]:
areas = pd.read_csv(ADDITIONAL_DATA + 'area_20230808.csv')
areatypes = pd.read_csv(ADDITIONAL_DATA + 'areatype_20230808.csv')
buildings = pd.read_csv(ADDITIONAL_DATA + 'building_20230808.csv')
districts = pd.read_csv(ADDITIONAL_DATA + 'district_20230808.csv')
geonims = pd.read_csv(ADDITIONAL_DATA + 'geonim_20230808.csv')
geonim_types = pd.read_csv(ADDITIONAL_DATA + 'geonimtype_20230808.csv')
prefixes = pd.read_csv(ADDITIONAL_DATA + 'prefix_20230808.csv')
subrfs = pd.read_csv(ADDITIONAL_DATA + 'subrf_20230808.csv')
towns = pd.read_csv(ADDITIONAL_DATA + 'town_20230808.csv')

C:\Users\saifu\AppData\Local\Temp\ipykernel_15316\786981087.py:3: DtypeWarning: Columns (4,6,7,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  buildings = pd.read_csv(ADDITIONAL_DATA + 'building_20230808.csv')


In [4]:
additional_data = [areas, areatypes, buildings, districts, geonims,geonim_types, prefixes, subrfs, towns]

In [5]:
df1 = pd.read_csv(DATA_PATH + 'dataset_1.csv')
df2 = pd.read_csv(DATA_PATH + 'dataset_2.csv')
df3 = pd.read_csv(DATA_PATH + 'dataset_3.csv')
df4 = pd.read_csv(DATA_PATH + 'dataset_4.csv')
df5 = pd.read_csv(DATA_PATH + 'dataset_5.csv')

In [6]:
cols_fine_tune = ['name', 'short_name', 'only_name']
data_2finetune = []

In [7]:
for df in additional_data:
    for col in cols_fine_tune:
        if col in df.columns:
            data_2finetune.append(df[col].values.tolist())

In [8]:
tokens = []
for sublist in data_2finetune:
    for item in sublist:
        tokens.append(item)

In [10]:
embedders_names = ['cointegrated/rubert-tiny', 'DeepPavlov/rubert-base-cased']

In [11]:
tokenizer = AutoTokenizer.from_pretrained(embedders_names[1])
model = AutoModel.from_pretrained(embedders_names[1])
new_tokens = set(tokens) - set(tokenizer.vocab.keys())

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
new_tokens = [AddedToken(str(token)) for token in new_tokens]

In [13]:
tokenizer.add_tokens(list(new_tokens))
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 158674. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(158674, 768)

In [14]:
model.save_pretrained('bert_new_vocab')

In [31]:
bert_model = BertModel.from_pretrained('bert_new_vocab')